 <img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/01-nvidia-logo-horiz-500x200-2c50-d@2x.png" width=300>
 
# 계산과학공학회 인공지능 겨울학교 2022
# [KSCSE](http://www.cse.or.kr/) 2022  GPU Tutorial  @ High1
# Day1 - Introducion to AI 
by Hyungon Ryu | NVAITC(NVIDIA AI Tech. Center)  Korea 


![](http://www.cse.or.kr/assets/img/logo_cse.png)

# Part 2 -  CNN Primer and Keras  


<br>
In this notebook you will be introduced to the concept of a convolutional neural network (CNN) and implement one using Keras. This notebook is designed as a starting point for absolute beginners to deep learning.

In this notebook you will be introduced to the concept of a convolutional neural network (CNN) and implement one using Keras. This notebook is designed as a starting point for absolute beginners to deep learning.

### Contents of this notebook:

 - How a deep learning project is planned
 - Wrapping things up with an example (classification)
  - Fully connected networks

### By the end of this notebook the participant will:

 - Understand machine learning pipelines
 - Write a deep learning classifier and train it


## Machine Learning Pipeline
During the bootcamp we will be making use of the following concepts to help us understand how a machine learning (ML) project should be planned and executed:

 1. <B>Data</B>: To start any ML project we need data which is pre-processed and can be fed into the network.
 2. <B>Task</B>: There are many possible tasks in the field of ML; we need to make sure we understand and define the problem statement accurately.
 3. <B>Model</B>: We need to build our model, which is neither too deep (requiring a lot of computational power) nor too small (preventing it from learning the important features).
 4. <B>Loss</B>: Out of the many loss functions that can be defined, we need to carefully choose one which is suitable for the task we are about to carry out.
 5. <B>Learning</B>: There are a variety of optimisers, each with their advantages and disadvantages. We must choose one which is suitable for our task and train our model using some suitably chosen hyperparameters.
 6. <B>Evaluation</B>: We must determine if our model has learned the features properly by analysing how it performs on data it has not previously seen.

### Step by Step  DL training 

#### load keras module

In [ ]:
# Import Necessary Libraries

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

#### prepare MNIST dataset

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

![](https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png)

Loading the dataset returns four NumPy arrays:

The `x_train` and `y_train` arrays are the training set—the data the model uses to learn.
The model is tested against the test set, the `x_test`, and `y_test` arrays.

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
# the data, split between train and validation sets
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

#### Understanding the Data

In [ ]:
# Print array size of training dataset
print("Size of Training Images: " + str(x_train.shape))
# Print array size of labels
print("Size of Training Labels: " + str(y_train.shape))

# Let's see how our outputs look
print("Training Set Labels: " + str(y_train))


In [ ]:
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
import matplotlib.pyplot as plt
image = x_train[0]
plt.imshow(image, cmap='gray')

In [ ]:
# Let's print to verify whether the data is of the correct format.
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

#### Data Preprocessing
 1d input
 for simplicity, flatting the 2D input to 1D array with `reshape` function

In [ ]:
x_train = x_train.reshape(60000, 784) # 28x28 = 784
x_valid = x_valid.reshape(10000, 784)

In [ ]:
print(x_train.shape)

The image pixel values range from 0 to 255. Let us now normalise the data range from 0 - 255 to 0 - 1 in both the Train and Test set. This normalisation of pixels helps us by optimizing the process where the gradients are computed.

In [ ]:
x_train = x_train / 255
x_valid = x_valid / 255 

In [ ]:
import tensorflow.keras as keras
num_categories = 10

#y_train = keras.utils.to_categorical(y_train, num_categories)
#y_valid = keras.utils.to_categorical(y_valid, num_categories)

# Building a Neuron

Neurons are the fundamental building blocks to a neural network. Just like how biological neurons send an electrical impulse under specific stimuli, artificial neural networks similarly result in a numerical output with a given numerical input.

We can break down building a neuron into 3 steps:

 - Defining the architecture
 - Intiating training ( compile)
 - Evaluating the model

![](https://camo.githubusercontent.com/b1cabba25cf7982d07a2a8ad60f344a0a69b463a75896d03c0e05ee02253a3bc/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f7468756d622f312f31302f426c617573656e5f303635375f4d756c7469706f6c61724e6575726f6e2e706e672f35313270782d426c617573656e5f303635375f4d756c7469706f6c61724e6575726f6e2e706e67)

Image courtesy of Wikimedia Commons

Biological neurons transmit information with a mechanism similar to Morse Code. It receives electrical signals through the dendrites, and under the right conditions, sends an electrical impulse down the axon and out through the terminals.

It is theorized the sequence and timing of these impulses play a large part of how information travels through the brain. Most artificial neural networks have yet to capture this timing aspect of biological neurons, and instead emulate the phenomenon with simpler mathematical formulas.

# The Math
Computers are built with discrete 0s and 1s whereas humans and animals are built on more continuous building blocks. Because of this, some of the first neurons attempted to mimic biological neurons with a linear regression function: $y = mx + b$. The $x$ is like information coming in through the dendrites and the $y$ is like the output through the terminals. As the computer guesses more and more answers to the questions we present it, it will update its variables ($m$ and $b$) to better fit the line to the data it has seen.

Neurons are often exposed to multivariate data. We're going to build a neuron that takes each pixel value (which is between `0` and `255`), and assign it a weight, which is equivalent to our m. Data scientists often express this weight as w. For example, the first pixel will have a weight of `w0`, the second will have a weight of `w1`, and so on. Our full equation becomes `y = w0x0 + w1x1 + w2x2 + ... + b`. 

Each image is 28 pixels by 28 pixels, so we will have a total of 784 weights. A pixel value of `0` would be black and a pixel value of `255` would be white. Let's look at the raw pixel values of the previous image we plotted. Each number below will be assigned a weight.

#### Defining our model

Our model has three layers:

 - 784 input features (28 * 28)
 - 512 nodes in the hidden layer (feel free to experiment with this value)
 - 512 nodes in the hidden layer (feel free to experiment with this value)
 - 10 output nodes to denote the class

We assume the input is 1d array. the network consists of a sequence of two tf.keras.layers.Dense layers. These are densely connected, or fully connected, neural layers. The first Dense layer has 128 nodes (or neurons). The second (and last) layer is a 10-node softmax layer that returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 classes.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(784,)))

In [ ]:
model.summary()

In [ ]:
model.add(Dense(units = 512, activation='relu'))

In [ ]:
model.summary()

In [ ]:
model.add(Dense(units = 10, activation='softmax'))

In [ ]:
model.summary()

#### Compile the model


Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

 - <B>Loss function</B> —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction. See KERAS's [loss functions](https://keras.io/api/losses/) section
 - <B>Optimizer</B> —This is how the model is updated based on the data it sees and its loss function. See Keras [Optimizer](https://keras.io/api/optimizers/) Section
 - <B>Metrics</B> —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified. See Keras's [Metrics](https://keras.io/api/metrics/) section

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In this case, we're going to use a type of function specific to classification called SparseCategoricalCrossentropy:

 - Sparse - for this function, it refers to how our label is an integer index for our categories
 - Categorical - this function was made for classification
 - Cross-entropy - the more confident our model is when it makes an incorrect guess, the worse its score will be. If a model is 100% confident when it is wrong, it will have a score of negative infinity!
from_logits - the linear output will be transformed into a probability which can be interpreted as the model's confidence that a particular category is the correct one for the given input.

#### train the model

Training the neural network model requires the following steps:

 1. Feed the training data to the model. In this example, the training data is in the `x_train`  and `y_train` arrays.
 2. The model learns to associate images and labels.
 3. You ask the model to make predictions about a test set—in this example, the   `x_test` array. Verify that the predictions match the labels from the `y_test` array.
 To start training, call the model.fit method—so called because it "fits" the model to the training data:

In [ ]:
hist = model.fit(
    x_train, y_train, epochs=5, verbose=1
)

In [ ]:
plt.plot(hist.history['loss'])

#### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(x_valid,  y_valid, verbose=2)

We get an accuracy of 97% on the test dataset, which is less than the 98.9% we got during the training phase. This problem in machine learning is called overfitting.

In [ ]:
# Making predictions from the test_images

predictions = model.predict(x_valid)

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h)
x_valid = x_valid.reshape(x_valid.shape[0], w, h)


# Helper functions to plot images 
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_valid, x_valid)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_valid)
plt.tight_layout()
plt.show()

## one hot encoding

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
# the data, split between train and validation sets
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(60000, 784)
x_valid = x_valid.reshape(10000, 784)

In [ ]:
x_train = x_train / 255
x_valid = x_valid / 255 

In [ ]:
import tensorflow.keras as keras
num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_valid_org = y_valid
y_valid = keras.utils.to_categorical(y_valid, num_categories)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(784,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = 10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(
    x_train, y_train, epochs=5, verbose=1
)

In [ ]:
plt.plot(hist.history['loss'])

In [ ]:
test_loss, test_acc = model.evaluate(x_valid,  y_valid, verbose=2)

In [ ]:
# Making predictions from the test_images

predictions = model.predict(x_valid)

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h)
x_valid = x_valid.reshape(x_valid.shape[0], w, h)


# Helper functions to plot images 
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_valid_org, x_valid) ## no one hot encoding for viz
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_valid_org)
plt.tight_layout()
plt.show()

## same with Fashion MNIST dataset

## Image classification on types of clothes


We will be using the Fashion MNIST dataset, which is a very popular introductory dataset in deep learning. This dataset contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels).

![fashin mnist](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/fashion-mnist.png)

#### prepare dataset

In [ ]:
# Let's Import the Dataset
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Loading the dataset returns four NumPy arrays:

 - The `train_images` and `train_labels` arrays are the training set—the data the model uses to learn.
 - The model is tested against the test set, the `test_images`, and `test_labels` arrays.


#### understand dataset

In [ ]:
# Print array size of training dataset
print("Size of Training Images: " + str(train_images.shape))
# Print array size of labels
print("Size of Training Labels: " + str(train_labels.shape))

# Print array size of test dataset
print("Size of Test Images: " + str(test_images.shape))
# Print array size of labels
print("Size of Test Labels: " + str(test_labels.shape))

# Let's see how our outputs look
print("Training Set Labels: " + str(train_labels))
# Data in the test dataset
print("Test Set Labels: " + str(test_labels))

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap='gray')
plt.grid(False)
plt.show()


The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the class of clothing the image represents:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Let's print to verify whether the data is of the correct format.
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

#### preprocessing

The image pixel values range from 0 to 255. Let us now normalise the data range from 0 - 255 to 0 - 1 in both the Train and Test set. This normalisation of pixels helps us by optimizing the process where the gradients are computed.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

#### define model

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

#### compile model

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(train_images, train_labels, epochs=5)

In [ ]:
plt.plot(hist.history['loss'])

#### Evaluate accuracy

In [ ]:
test_loss, test_acc = model.evaluate(test_images ,  test_labels, verbose=2)

In [ ]:
# Making predictions from the test_images

predictions = model.predict(test_images)

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
train_images = train_images.reshape(train_images.shape[0], w, h)
test_images = test_images.reshape(test_images.shape[0], w, h)


# Helper functions to plot images 
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

# end of jupyter part2
### Navigation  |[Part1-reg](part1_LR.ipynb) |  [Part2-MLP](part2_MLP.ipynb) |  [Part3-CNN](part3_CNN.ipynb) |  [Part4-ResNet](part4_resnet.ipynb) | [Part5-MLP Mixer](part5_Mixer.ipynb) |

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/01-nvidia-logo-horiz-500x200-2c50-d@2x.png" width=300>